In [41]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=20

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=5
p=floor(Int64,n/u)+1

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
f=sum(generate_random_poly(I[j]) for j in 1:p)



g=-(x.^2).+(1/u)



m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)

randx=2*rand(n).-1
randx=randx./sqrt(u)

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)
println("====================")

k=2

println("Relaxed order: k=",k)
println("====================")

t=1

println("Sparse order: t=",t)
println("====================")

***Problem setting***
Number of variable: n=20
Number of inequality constraints: m=20
Number of equality constraints: l=3
Relaxed order: k=2
Sparse order: t=1


In [42]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=true);

  likely near /home/hoanganh/Desktop/math-topics/ctpPOP/codes/ctpPOP/solvers/ProximalBundleMethod/ProximalMethod.jl:391


In [43]:
using TSSOS


@time opt,sol,data=cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS="block",CTP=false);

------------------------------------------------------
The clique sizes of varibles:
[6, 5]
[3, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 541             
  Cones                  : 0               
  Scalar variables       : 87              
  Matrix variables       : 32              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 

In [46]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_mix_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,EigAlg="Arpack",maxit=1e10,tol=1e-2,UseEq=false)

------------------------------------------------------
The clique sizes of varibles:
[6, 5]
[3, 1]
------------------------------------------------------


  likely near /home/hoanganh/Desktop/math-topics/ctpPOP/codes/ctpPOP/solvers/ProximalBundleMethod/ProximalMethod.jl:391


  Number of cliques: p=4
  Largest clique size: u=6
  Largest constant trace: a=3.6399999999999997
  Number of blocks: omega=54
  Number of equality consraints: zeta=995
  Size of the largest block: s^max=22
Modeling time:
  7.937845 seconds (4.33 M allocations: 218.481 MiB, 1.56% gc time)
iter=1.0   val=-16.919549937125073   gap=0.0   feas=7.741025621337651
iter=2.0   val=-5.639849979041691   gap=12.976702645470773   feas=1.8143477849511975
iter=4.0   val=-5.122174640379852   gap=6.759798506739603   feas=2.4756476775736087
iter=8.0   val=-4.932596954266475   gap=2.9886663663867914   feas=1.3773610180978095
iter=16.0   val=-3.1747095625176955   gap=2.616158202673109   feas=0.8672726534004706
iter=32.0   val=-3.3321077759527062   gap=2.258863523242298   feas=0.4059736854775491
iter=64.0   val=-3.1621876607367447   gap=1.3943085609233261   feas=0.24687399574393917
iter=128.0   val=-3.1983667905034205   gap=0.8454047801246882   feas=0.1261396307262505
iter=256.0   val=-3.27808670396499   

-3.257407735486295